here, i i will attempt to find the bug that is preventing certain protein mutations from running quickly...

In [1]:
import simtk.openmm as openmm
from openmmtools.constants import kB
import simtk.unit as unit
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.rjmc.topology_proposal import TopologyProposal, NetworkXMolecule
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from openmmtools import mcmc, utils
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
#cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from perses.annihilation.lambda_protocol import LambdaProtocol
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
import openmmtools.integrators as integrators
from protein_test_cycle import *

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [2]:
def create_hss(reporter_name, hybrid_factory, selection_string ='all', checkpoint_interval = 1, n_states = 13):
    lambda_protocol = LambdaProtocol(functions='default')
    reporter = MultiStateReporter(reporter_name, analysis_particle_indices = hybrid_factory.hybrid_topology.select(selection_string), checkpoint_interval = checkpoint_interval)
    hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                 collision_rate=5.0 / unit.picosecond,
                                                                                 n_steps=250,
                                                                                 reassign_velocities=False,
                                                                                 n_restart_attempts=20,
                                                                                 splitting="V R R R O R R R V",
                                                                                 constraint_tolerance=1e-06),
                                                                                 hybrid_factory=hybrid_factory, online_analysis_interval=10)
    hss.setup(n_states=n_states, temperature=300*unit.kelvin,storage_file=reporter,lambda_protocol=lambda_protocol,endstates=False)
    return hss, reporter

In [3]:
def deserialize(filename):
    import pickle
    with open(filename, 'rb') as f:
        htf = pickle.load(f)
    return htf



In [4]:
fast_htf = deserialize('ALA_SER.sol.pkl')
slow_htf = deserialize('ALA_ILE.sol.pkl') 
fast_hss, _ = create_hss(f"fast_debug.nc", fast_htf)
slow_hss, _ = create_hss(f"slow_debug.nc", slow_htf)

/home/rufad/miniconda3/envs/openmm/lib/python3.6/site-packages/openmmtools-0.18.3+0.g6cf88cc.dirty-py3.6-linux-x86_64.egg/openmmtools/multistate/multistatereporter.py:116: UserWarning: Warning: The openmmtools.multistate API is experimental and may change in future releases
  warnings.warn('Warning: The openmmtools.multistate API is experimental and may change in future releases')
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as fast_debug_checkpoint.nc
/home/rufad/miniconda3/envs/openmm/lib/python3.6/site-packages/openmmtools-0.18.3+0.g6cf88cc.dirty-py3.6-linux-x86_64.egg/openmmtools/multistate/multistatesampler.py:145: UserWarning: Warning: The openmmtools.multistate API is experimental and may change in future releases
  warnings.warn('Warning: The openmmtools.multistate API is experimental and may change in future releases')
INFO:perses.samplers.multistate:n_replicas not defined, setting to match n_states, 13
DEBUG:mpiplus.mpiplus:Cann

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as slow_debug_checkpoint.nc
INFO:perses.samplers.multistate:n_replicas not defined, setting to match n_states, 13
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b9267b87550>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b925440e0d0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b92543cd2f0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b92543cd598>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  44445B | 43.403KB | 0.042MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.107s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange o

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [6]:
fast_hss.extend(2)
print(f"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
slow_hss.extend(2)

DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateSampler._store_options of <instance of HybridRepexSampler>>
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 2/3
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b9254413d08>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.003s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 16158/57122 attempted swaps (28.3%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:m

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   10.416s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.122s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b92543cd6a8>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b92543cd950>
DEBUG:openmmtools.utils:Storing sampler states took    0.010s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.038s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b92543d0488>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesam

In [ ]:
# fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(fast_htf.hybrid_system)
# fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))

# slow_lambda_alchemical_state = RelativeAlchemicalState.from_system(slow_htf.hybrid_system)
# slow_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))

# fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(fast_htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
# slow_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(slow_htf.hybrid_system, temperature = temperature),composable_states = [slow_lambda_alchemical_state])

# fast_sampler_state = SamplerState(positions = fast_htf._hybrid_positions, box_vectors=fast_htf.hybrid_system.getDefaultPeriodicBoxVectors())
# slow_sampler_state = SamplerState(positions = slow_htf._hybrid_positions, box_vectors=slow_htf.hybrid_system.getDefaultPeriodicBoxVectors())

# integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
#                                                  timestep = 4.0* unit.femtoseconds,
#                                                  splitting = 'V R O R V',
#                                                  measure_shadow_work = False,
#                                                  measure_heat = False,
#                                                  constraint_tolerance = 1e-6,
#                                                  collision_rate = 5.0 / unit.picoseconds)

# integrator_2 = integrators.LangevinIntegrator(temperature = temperature,
#                                                  timestep = 4.0* unit.femtoseconds,
#                                                  splitting = 'V R O R V',
#                                                  measure_shadow_work = False,
#                                                  measure_heat = False,
#                                                  constraint_tolerance = 1e-6,
#                                                  collision_rate = 5.0 / unit.picoseconds)


# fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
# slow_context, slow_integrator = cache.global_context_cache.get_context(slow_thermodynamic_state, integrator_2)

# fast_sampler_state.apply_to_context(fast_context)
# slow_sampler_state.apply_to_context(slow_context)

perhaps if we remove the constraints, it will run faster?
